In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

def load_and_split_motor_data(base_path, folders, test_size=0.2):
    # 데이터를 저장할 리스트 초기화
    all_data = {'normal': [], 'abnormal': []}

    # 정상 및 비정상 데이터 처리
    for condition in ['normal', 'abnormal']:
        condition_path = os.path.join(base_path, condition)
        
        # 모든 폴더 순회
        for number in folders:
            numbers_path = os.path.join(condition_path, str(number))
            final_path = os.path.join(numbers_path, 'time_vib')
            
            # 폴더 내 CSV 파일 목록 가져오기
            if os.path.exists(final_path):  # 폴더가 존재하는지 확인
                csv_files = [os.path.join(final_path, file) for file in os.listdir(final_path) if file.endswith('.csv')]
                
                for csv_file in csv_files:
                    # CSV 데이터 읽기
                    df = pd.read_csv(csv_file, header=None, names=['time', 'vibration'])
                    all_data[condition].append(df['vibration'].values)  # 진동 데이터 저장

    # 데이터를 합치고 split
    normal_data = pd.DataFrame({'vibration': all_data['normal']})
    abnormal_data = pd.DataFrame({'vibration': all_data['abnormal']})

    # Train/Test Split
    x_train_normal, x_test_normal = train_test_split(normal_data['vibration'], test_size=test_size, random_state=42)
    x_train_abnormal, x_test_abnormal = train_test_split(abnormal_data['vibration'], test_size=test_size, random_state=42)

    # 결과 반환
    return {
        'train': {'normal': x_train_normal, 'abnormal': x_train_abnormal},
        'test': {'normal': x_test_normal, 'abnormal': x_test_abnormal}
    }

# 폴더 번호 정의
folders = list(range(1, 21))  # 1~20번 폴더 모두 사용

# 데이터 로드 및 분할
motor_data = load_and_split_motor_data("./motor_data", folders)

# Train/Test 데이터 출력    
x_train_normal = motor_data['train']['normal']
x_train_abnormal = motor_data['train']['abnormal']
x_test_normal = motor_data['test']['normal']
x_test_abnormal = motor_data['test']['abnormal']

# 출력 예시
print(f"Train Normal Data: {len(x_train_normal)} sequences")
print(f"Train Abnormal Data: {len(x_train_abnormal)} sequences")
print(f"Test Normal Data: {len(x_test_normal)} sequences")
print(f"Test Abnormal Data: {len(x_test_abnormal)} sequences")


Train Normal Data: 1661 sequences
Train Abnormal Data: 1665 sequences
Test Normal Data: 416 sequences
Test Abnormal Data: 417 sequences


In [ ]:
import os

def load(base_path, state, numbers):
    data = []
    # 경로 생성
    final_path = os.path.join(base_path, state, numbers, "time_vib")  # "time_vib"를 직접 입력
    
    csv_files = [os.path.join(final_path, file) for file in os.listdir(final_path) if file.endswith('.csv')]
                
    for csv_file in csv_files:
        # CSV 데이터 읽기
        df = pd.read_csv(csv_file, header=None, names=['time', 'vibration'])
        data[state][numbers].append(df['vibration'].values)  # 진동 데이터 저장
    return number_path



# 함수 호출
a=load("./motor_data", "normal", '1')
print(a)

./motor_data/normal/1/time_vib
./motor_data/normal/1/time_vib


In [90]:
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
scaled_data = scaler.fit_transform(x_train_normal)
print(scaled_data)

ValueError: setting an array element with a sequence.

In [75]:
import numpy as np

# 데이터를 NumPy 배열로 변환
x_train_normal = np.array([np.array(seq) for seq in x_train_normal], dtype=object)
x_test_normal = np.array([np.array(seq) for seq in x_test_normal], dtype=object)

# 길이가 고정되지 않은 데이터를 패딩 처리하거나 샘플링
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train_normal = pad_sequences(x_train_normal, maxlen=100, padding='post')  # 최대 길이 100으로 패딩
x_test_normal = pad_sequences(x_test_normal, maxlen=100, padding='post')

# 정규화
x_train_normal = x_train_normal / np.max(x_train_normal)
x_test_normal = x_test_normal / np.max(x_test_normal)


/tmp/ipykernel_16227/1190269993.py:14: RuntimeWarning: invalid value encountered in divide
  x_train_normal = x_train_normal / np.max(x_train_normal)
/tmp/ipykernel_16227/1190269993.py:15: RuntimeWarning: invalid value encountered in divide
  x_test_normal = x_test_normal / np.max(x_test_normal)


In [76]:
# 모델 정의
input_dim = x_train_normal.shape[1]  # 데이터의 차원 (100으로 패딩된 길이)
encoding_dim = 32  # 잠재 공간 크기
autoencoder = build_autoencoder(input_dim, encoding_dim)

# 모델 학습
history = autoencoder.fit(
    x_train_normal, x_train_normal,  # 입력과 출력이 동일
    epochs=50,
    batch_size=32,
    validation_data=(x_test_normal, x_test_normal)
)


Epoch 1/50


2024-11-24 20:06:16.141971: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: nan - val_loss: nan
Epoch 2/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 3/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
Epoch 4/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 5/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 6/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
Epoch 7/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 8/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 9/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 10/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: nan - val_loss: nan
Epoch 11/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 12/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: nan - val_loss: nan
Epoch 13/50
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 988us/step - loss: nan - val_l